In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

os.chdir('/Users/reejungkim/Documents/Git/working-in-progress')
load_dotenv()
key = os.getenv('huggingface_read')


client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ[key],
)

completion = client.chat.completions.create(
    model="openai/gpt-oss-20b:novita",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
)

In [ ]:
from transformers import pipeline
import torch

model_id = "openai/gpt-oss-20b"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype="auto",
    device_map="auto",
)

messages = [
    {"role": "user", "content": "Explain quantum mechanics clearly and concisely."},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])
